In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import pymongo
import pyarrow as pa
import pyarrow.parquet as pq
from pymongoarrow.monkey import patch_all
patch_all()

from config import Config

client = pymongo.MongoClient(Config.MONGO_URI)
db = client[Config.MONGO_DB]

In [22]:
# aggregate + save
from loader import run
need_to_rebuild_parquets = False # set to True to rebuild parquet files from DB
if need_to_rebuild_parquets:
    run()

In [38]:
# load
phishing = pq.read_table('floor/phishing.parquet')
benign_u = pq.read_table('floor/benign.parquet')

from transformers.drop_nontrain import drop_nontrain
phishing = drop_nontrain(phishing)
benign_u = drop_nontrain(benign_u)

# realign schemas (parquet files save in nonsense orders)
phishing_order = phishing.schema.names
benign = benign_u.select(phishing_order)

data = pa.concat_tables([phishing, benign])
df = data.to_pandas()
from transformers.cast_timestamp import cast_timestamp
df = cast_timestamp(df)
df

import pandas as pd


pd.set_option('display.max_rows', 2000)

# Drop unseless / problematic features
df = df.dropna(subset = ['tmp_tld', 'tmp_sld', 'tmp_stld', 'tmp_concat_subdomains'], inplace = False)
df = df.drop(['tmp_tld', 'tmp_sld', 'tmp_stld', 'tmp_concat_subdomains'], axis=1, inplace = False)



"""
df = df.drop(['domain_registration_date', 'domain_expiration_date', 'domain_last_changed_date'], axis=1, inplace = False)
"""

df


,label,average_rtt,has_tls,tls_chain_len,tls_version_score,tls_cipher_score,root_cert_validity_len,root_cert_validity_remaining,leaf_cert_validity_len,leaf_cert_validity_remaining,...,domain_lifetime,domain_time_from_last_change,dns_soa_primary_ns_subdomains,dns_soa_primary_ns_digit_count,dns_soa_primary_ns_len,dns_soa_admin_email_len,dns_soa_admin_email_subdomains,dns_soa_admin_email_digit_count,domain_name_in_mx,dns_txt_google_vetified
0,misp:phishing,98.4360,True,4.0,2.0,0.0,9066.0,4129.0,345.0,329.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,misp:phishing,0.0000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,misp:phishing,-1.0000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,misp:phishing,0.0000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,misp:phishing,0.0000,True,3.0,3.0,1.0,3583.0,2124.0,396.0,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295348,benign:unknown,3.7640,True,4.0,3.0,2.0,9132.0,2124.0,366.0,327.0,...,NaN,NaN,0.0,0.0,23.0,19.0,0.0,0.0,0,0
295349,benign:unknown,6.4805,True,3.0,3.0,2.0,1349.0,570.0,90.0,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
295350,benign:unknown,15.4715,True,3.0,3.0,2.0,1349.0,570.0,90.0,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
295351,benign:unknown,94.7250,True,3.0,2.0,0.0,1349.0,570.0,90.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [62]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import datetime

#le = LabelEncoder()
#labels = le.fit_transform(df['label'])

class_map = {"benign:unknown": 0, "misp:phishing": 1}
labels = df['label'].apply(lambda x: class_map[x])

features = df.drop('label', axis=1).copy()

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42, shuffle=True, stratify=labels)

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

start = datetime.datetime.now()
"""
params = {
    "max_depth": 9,
    "eta": 0.2,
    "objective": "binary:logistic",
    #"num_class": 2,
    "tree_method": "gpu_hist",
    "min_child_weight": 1.7,
    "subsample": 0.4,
    "sampling_method": "gradient_based"
}
"""
params = {
    "max_depth": 9,
    "eta": 0.2,
    "objective": "binary:logistic",
    #"num_class": 2,
    "tree_method": "gpu_hist",
    "min_child_weight": 2.2,
    "subsample": 0.7,
    "sampling_method": "gradient_based"
}
trees = 250

model = XGBClassifier(
  **params, n_estimators=trees
)
do_cross_val = True
if do_cross_val:
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
  results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='f1')
  print("F1: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

finish = datetime.datetime.now()
delta = finish - start
print(delta)

F1: 93.31% (0.30%)
0:00:18.579605


In [30]:
## Hyperparameter tuning

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import datetime

class_map = {"benign:unknown": 0, "misp:phishing": 1}


labels = df['label'].apply(lambda x: class_map[x])
features = df.drop('label', axis=1).copy()

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42, shuffle=True, stratify=labels)

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

params = {
    "eta": 0.2,
    "objective": "binary:logistic",
    #"num_class": 2,
    "tree_method": "gpu_hist",
    #"min_child_weight": 1,
    #"subsample": 0.4,
    #"sampling_method": "gradient_based",
    #"predictor": "gpu_predictor"
}
trees = 250

clf = XGBClassifier(**params, n_estimators=trees)

grid = {
    #'max_depth': [5, 6, 7, 8, 9, 10, 12, 15, 17, 20, 25, 30, 40, 50],
    #'max_depth': [8, 9, 10, 12, 15],
    'max_depth': [9],
    #"max_depth": [10],
    "min_child_weight": [1, 1.2, 1.5, 1.7, 2.0, 2.2, 2.5],
    #"min_child_weight": [1],
    #"subsample": [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    #"sampling_method": ["uniform", "gradient_based"]
    "sampling_method": ["gradient_based"],
    #"subsample": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    "subsample": [0.4],
    
}


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

grid_search = GridSearchCV(estimator=clf,
                           param_grid=grid,
                           #n_jobs=-1,
                           cv=cv,
                           scoring='neg_log_loss',#'f1',
                           verbose=3,
                           return_train_score=True)

grid_search.fit(X_train, y_train)

clf = grid_search.best_estimator_

scores = pd.DataFrame(grid_search.cv_results_)
col_names = ['mean_train_score', 'mean_test_score']

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV 1/5] END max_depth=9, min_child_weight=1, sampling_method=gradient_based, subsample=0.4;, score=(train=-0.014, test=-0.047) total time=   3.7s
[CV 2/5] END max_depth=9, min_child_weight=1, sampling_method=gradient_based, subsample=0.4;, score=(train=-0.014, test=-0.046) total time=   3.7s
[CV 3/5] END max_depth=9, min_child_weight=1, sampling_method=gradient_based, subsample=0.4;, score=(train=-0.014, test=-0.046) total time=   3.7s
[CV 4/5] END max_depth=9, min_child_weight=1, sampling_method=gradient_based, subsample=0.4;, score=(train=-0.013, test=-0.045) total time=   3.8s
[CV 5/5] END max_depth=9, min_child_weight=1, sampling_method=gradient_based, subsample=0.4;, score=(train=-0.013, test=-0.045) total time=   3.7s
[CV 1/5] END max_depth=9, min_child_weight=1.2, sampling_method=gradient_based, subsample=0.4;, score=(train=-0.015, test=-0.047) total time=   3.7s
[CV 2/5] END max_depth=9, min_child_weight=1.2, sampling

In [61]:
scores = pd.DataFrame(grid_search.cv_results_)

from numpy import array
from numpy.ma import masked_array
from numpy import *

col_names = ['mean_train_score', 'mean_test_score']
means_df = scores[col_names]
ax = means_df.plot(kind='line', grid=True)


from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [12, 12]
plt.rcParams["figure.autolayout"] = True

#plt.rcParams["figure.dpi"] = 800

max_ids = means_df.idxmax(axis=0)

for i in range(len(max_ids)):
    for col_name in col_names:
        value = means_df[col_name][max_ids[i]]
        id = max_ids[i]
        
        color = 'r' if max_ids[i] == max_ids['mean_test_score'] else 'grey'
        
        ax.scatter([id], [value],
                   marker='o',
                   color=color,
                   label='point',)

        ax.annotate(str(round(value, 3))+"-"+str(id),
                    (id, value),
                    xytext=(id+3, value))
        

In [46]:

scores

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_child_weight,param_sampling_method,param_subsample,params,split0_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,3.528571,0.036520,0.179273,0.009234,9,1,gradient_based,0.4,"{'max_depth': 9, 'min_child_weight': 1, 'sampl...",-0.047369,...,-0.045913,0.000824,7,-0.013517,-0.014487,-0.013623,-0.013167,-0.013420,-0.013643,0.000448
1,3.522574,0.047046,0.171906,0.001219,9,1.2,gradient_based,0.4,"{'max_depth': 9, 'min_child_weight': 1.2, 'sam...",-0.046576,...,-0.045536,0.000876,2,-0.014555,-0.014717,-0.014762,-0.013469,-0.013653,-0.014231,0.000555
2,3.481111,0.055535,0.169356,0.002307,9,1.5,gradient_based,0.4,"{'max_depth': 9, 'min_child_weight': 1.5, 'sam...",-0.047033,...,-0.045537,0.001076,3,-0.014564,-0.015195,-0.014528,-0.015469,-0.014406,-0.014833,0.000420
3,3.473173,0.046598,0.167395,0.001710,9,1.7,gradient_based,0.4,"{'max_depth': 9, 'min_child_weight': 1.7, 'sam...",-0.047267,...,-0.045707,0.000886,4,-0.014912,-0.015038,-0.014624,-0.015238,-0.015342,-0.015031,0.000253
4,3.474966,0.033425,0.170659,0.003864,9,2.0,gradient_based,0.4,"{'max_depth': 9, 'min_child_weight': 2.0, 'sam...",-0.047353,...,-0.045396,0.001070,1,-0.015889,-0.016154,-0.016184,-0.015776,-0.015845,-0.015970,0.000167
5,3.461393,0.056630,0.166856,0.002165,9,2.2,gradient_based,0.4,"{'max_depth': 9, 'min_child_weight': 2.2, 'sam...",-0.046976,...,-0.045791,0.000893,6,-0.016192,-0.016389,-0.015498,-0.015579,-0.015322,-0.015796,0.000417
6,3.454364,0.064021,0.166149,0.007814,9,2.5,gradient_based,0.4,"{'max_depth': 9, 'min_child_weight': 2.5, 'sam...",-0.046899,...,-0.045726,0.000953,5,-0.017131,-0.016814,-0.017308,-0.016766,-0.016753,-0.016955,0.000225


In [63]:
_ = model.fit(X_train, y_train)

In [64]:
predicted = model.predict(X_test)

In [65]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predicted))

0.9361228371294706


In [28]:
# get gain score
score = model.get_booster().get_score(importance_type='gain')
sorted_score = sorted(score.items(), key=lambda x: x[1], reverse=True)
sorted_score

[('has_tls', 122.59606170654297),
 ('joint-iso-itu_policy_count', 98.2258529663086),
 ('broken_chain', 65.35491180419922),
 ('dns_SOA_count', 60.26219940185547),
 ('dns_TXT_count', 48.61553192138672),
 ('lex_www_flag', 48.0981559753418),
 ('domain_lifetime', 47.015716552734375),
 ('lex_phishing_keyword_count', 43.551631927490234),
 ('root_cert_validity_remaining', 38.23231887817383),
 ('lex_digit_count', 33.647972106933594),
 ('lex_sub_count', 31.579769134521484),
 ('CA_certs_in_chain_ratio', 28.975114822387695),
 ('root_cert_validity_len', 27.736650466918945),
 ('tls_cipher_score', 26.709396362304688),
 ('dns_CNAME_count', 26.403310775756836),
 ('dns_A_count', 25.54125213623047),
 ('dns_NS_count', 22.961936950683594),
 ('domain_name_in_mx', 22.29218101501465),
 ('lex_subdomain_len', 19.874103546142578),
 ('client_auth_certs', 19.19256019592285),
 ('leaf_cert_validity_len', 17.810302734375),
 ('tls_version_score', 17.411319732666016),
 ('domain_registration_period', 17.223304748535156)

In [65]:
_ = model.fit(X_train, y_train)

In [66]:
predicted = model.predict(X_test)

In [67]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predicted))

0.935652013469256


In [19]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, predicted, labels=[1,0]))
print(classification_report(y_test, predicted, digits=4))


[[10089  1009]
 [  368 77140]]
              precision    recall  f1-score   support

           0     0.9871    0.9953    0.9912     77508
           1     0.9648    0.9091    0.9361     11098

    accuracy                         0.9845     88606
   macro avg     0.9759    0.9522    0.9636     88606
weighted avg     0.9843    0.9845    0.9843     88606



In [19]:
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
    
    #false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(y_test, y_score1)
    #false_positive_rate2, true_positive_rate2, threshold2 = roc_curve(y_test, y_score2)
    
    # Compute micro-average ROC curve and ROC area
    fpr, tpr, _ = roc_curve(y_test['label'].values, check)
    roc_auc = auc(fpr, tpr)
    #xgb.plot_importance(gbm)
    #plt.show()
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([-0.02, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()
    ##################################################

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices